In [1]:
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
import pandas as pd
from sklearn import svm
import numpy as np
import math

import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import colors
from sklearn.model_selection import train_test_split
from sklearn import datasets
from matplotlib.colors import ListedColormap
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedShuffleSplit,StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import GridSearchCV, LeaveOneOut, cross_val_predict
from sklearn.model_selection import KFold  
from sklearn.linear_model import LogisticRegression  
from sklearn.naive_bayes import GaussianNB  
from sklearn.neighbors import KNeighborsClassifier   
from sklearn import svm  
from sklearn.tree import DecisionTreeClassifier  
from sklearn.ensemble import RandomForestClassifier 
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_auc_score
import math
import datetime
import multiprocessing as mp
from sklearn.ensemble import StackingClassifier
from sklearn.pipeline import make_pipeline
from sklearn.svm import LinearSVC

import random


# In[25]:


# cores = mp.cpu_count()
# pool = mp.Pool(processes=20)

# min1=min(list(X_standard.shape))
# a1=list(range(10,min1,10))
# a1[-1]=min1
# pcc_top1=pool.map(find_best_pca_lr,a1)  ##（SVM，LR）
# pool.terminate()


# a1=pd.DataFrame()
# for i1 in pcc_top1:
#     a1=pd.concat([a1,i1], axis=0)
# a1.columns=['tr_TP', 'tr_TN', 'tr_FP', 'tr_FN', 'tr_Sen', 'tr_Spe', 'tr_Acc', 'tr_PPV', 'tr_NPV', 'tr_MCC', 'tr_AUC','TP',
#        'TN', 'FP', 'FN', 'Sen', 'Spe', 'Acc', 'PPV', 'NPV', 'MCC', 'AUC','PCA']   

# a1=a1.sort_values(['Acc','tr_Acc','PCA'], ascending=[False,False,True])

In [2]:
def read_single_csv(input_path,i):

    df_chunk=pd.read_csv(input_path,
                         chunksize=10000,
#                          sep='\t'
                        sep=i
                        )
    res_chunk=[]
    for chunk in df_chunk:
        res_chunk.append(chunk)
    res_df=pd.concat(res_chunk)
    return res_df

In [3]:
def get_measures_gridloo(label, score):
    label = np.array(label)
    score = np.array(score)
    
    N  = len(label)
    TP = sum((label == 1) & (score == 1))
    TN = sum((label == 0) & (score == 0))
    FP = sum((label == 0) & (score == 1))
    FN = sum((label == 1) & (score == 0))

    # init all measures to nan
    measures = {measure: float("nan") for measure in ("Sen", "Spe", "Acc", "PPV", "NPV", "MCC","AUC")}
    
    measures["TP"] = TP
    measures["TN"] = TN
    measures["FP"] = FP
    measures["FN"] = FN
    
    S = (TP + FN) / N
    P = (TP + FP) / N

    if (TP + FN) > 0:
        measures["Sen"] = round(TP/(TP+FN), 4)

    if (TN + FP) > 0:
        measures["Spe"] = round(TN/(TN+FP), 4)

    if (TP + FP + FN + TN) > 0:
        measures["Acc"] = round((TP+TN)/(TP+FP+FN+TN), 4)

    if (TP + FP) > 0:
        measures["PPV"] = round(TP/(TP+FP), 4)

    if (TN + FN) > 0:
        measures["NPV"] = round(TN/(TN+FN), 4)

    if (S*P*(1-S)*(1-P)) > 0:
        measures["MCC"] = round((TP/N - S*P)/(math.sqrt(S*P*(1-S)*(1-P))), 4)
    
    
    measures["AUC"]= roc_auc_score(label, score)
    return pd.DataFrame([measures],
                        columns=["TP", "TN", "FP",
                                 "FN", "Sen", "Spe", "Acc", "PPV", "NPV", "MCC","AUC"])

In [4]:
##SVM
def find_best_pca(i):
    n_components=i
    estimator = PCA(n_components=n_components,random_state=10)
    pca_X_train = estimator.fit_transform(X_standard)
    pca_X_test = estimator.transform(X_standard_test)
    
    scoring = {'AUC': 'roc_auc'}
    cost = [-5, -3, -1, 1, 3, 5, 7, 9, 11, 13, 15]
    gam = [3, 1, -1, -3, -5, -7, -9, -11, -13, -15]
    parameters =[{'kernel': ['rbf'], 'C': [2**x for x in cost],'gamma':[2**x for x in gam]}]
    
    cvx = StratifiedKFold(n_splits=5, shuffle=True, random_state=10) 
    grid_search=GridSearchCV(estimator=SVC(random_state=10),param_grid=parameters,cv=cvx,scoring='accuracy')
    grid_search.fit(pca_X_train, train_y)
    
    train_pre_y = cross_val_predict(grid_search.best_estimator_, pca_X_train,train_y, cv=cvx)
    train_res1=get_measures_gridloo(train_y,train_pre_y)
#     train_res1['PCA']=n_components
    test_pre_y = grid_search.predict(pca_X_test)
    test_res1=get_measures_gridloo(test_y,test_pre_y)
    test_res1['PCA']=n_components
    
    res=pd.concat([train_res1,test_res1], axis=1)
    return res

In [5]:
def find_best_pca_lr(i):
    n_components=i
    estimator = PCA(n_components=n_components,random_state=10)
    pca_X_train = estimator.fit_transform(X_standard)
    pca_X_test = estimator.transform(X_standard_test)
    
    cvx = StratifiedKFold(n_splits=5, shuffle=True, random_state=10) 

    param_grid = {'penalty':['l2'],
              "C":[0.00001,0.0001,0.001, 0.01, 0.1, 1, 10, 100, 1000],  
              "solver":["newton-cg", "lbfgs","liblinear"],
#             "C":list(np.arange(0,0.1,0.005)),   
#               "solver":["newton-cg", "lbfgs","liblinear"]
            }
    LR_grid = LogisticRegression()
    grid_search = GridSearchCV(LR_grid, param_grid=param_grid, cv=cvx ,scoring='accuracy',n_jobs=9)
    grid_search.fit(pca_X_train, train_y)


    train_pre_y = cross_val_predict(grid_search.best_estimator_, pca_X_train,train_y, cv=cvx)
    train_res1=get_measures_gridloo(train_y,train_pre_y)
#     train_res1['PCA']=n_components
   

    test_pre_y = grid_search.predict(pca_X_test)
    test_res1=get_measures_gridloo(test_y,test_pre_y)
    test_res1['PCA']=n_components
    
    res=pd.concat([train_res1,test_res1], axis=1)
    return res

In [133]:
cancer='BRCA'
path='D:\\lymph_meta_xmiseq\\tcga_data\\'
# e1="./"+cancer+"_delta_pcc_wilcox.csv"
# delta_pcc=read_single_csv(e1,',')
# delta_pcc=delta_pcc.drop(['Unnamed: 0','pair'],axis=1)
# delta_pcc=delta_pcc.dropna(axis=0,how='any')

# result=delta_pcc.T
# train=result.iloc[:,0:-1].values.astype('float')
# target=result.iloc[:,-1].values.astype('float')

train=pd.read_csv(path+cancer+"\\admat_final.csv").iloc[:,2:].T.values
target=pd.read_csv(path+cancer+"\\gctm_label.csv",index_col=0).values.ravel()

In [86]:
train

array([[ 0.00874834, -0.00736022, -0.01710605, ...,  0.0059139 ,
         0.00585509,  0.01760065],
       [-0.06756009, -0.03635654, -0.06104614, ...,  0.00184041,
         0.01073223, -0.00660949],
       [ 0.05017487, -0.03385318, -0.05728117, ..., -0.01067972,
         0.01398897,  0.01295414],
       ...,
       [-0.00012984,  0.00158047,  0.00504004, ...,  0.00526114,
        -0.00421565, -0.0168382 ],
       [-0.0120549 ,  0.0223974 , -0.02588812, ..., -0.03524445,
         0.00812885,  0.01039054],
       [ 0.00757404,  0.00548185,  0.02304116, ...,  0.00104388,
         0.00138618, -0.0349668 ]])

In [87]:
target

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [183]:
train_X,test_X, train_y, test_y = train_test_split(train,
                                                   target,
                                                   test_size = 0.3,
                                                   stratify=target,
                                                   random_state=10
                                                   )

standardScaler = StandardScaler()
standardScaler.fit(train_X)
X_standard = standardScaler.transform(train_X)
X_standard_test = standardScaler.transform(test_X)

n_components=0.95
estimator = PCA(n_components=n_components,random_state=10)
pca_X_train = estimator.fit_transform(X_standard)
pca_X_test = estimator.transform(X_standard_test)
cvx = StratifiedKFold(n_splits=5, shuffle=True, random_state=10) 

In [184]:
train_X.shape

(126, 2281)

In [185]:
#svm
scoring = {'AUC': 'roc_auc'}
cost = [-5, -3, -1, 1, 3, 5, 7, 9, 11, 13, 15]
gam = [3, 1, -1, -3, -5, -7, -9, -11, -13, -15]
parameters =[{'kernel': ['rbf'], 'C': [2**x for x in cost],'gamma':[2**x for x in gam]}]
cvx = StratifiedKFold(n_splits=5, shuffle=True, random_state=10) 

svc_grid_search=GridSearchCV(estimator=SVC(random_state=10),param_grid=parameters,cv=cvx,scoring='accuracy')
svc_grid_search.fit(pca_X_train, train_y)

#LR
param_grid = {'penalty':['l1', 'l2'],
              "C":[0.00001,0.0001,0.001, 0.01, 0.1, 1, 10, 100, 1000],
              "solver":["newton-cg", "lbfgs","liblinear","sag","saga"]
#               "algorithm":['auto', 'ball_tree', 'kd_tree', 'brute']
            }
LR_grid = LogisticRegression(max_iter=1000)
LR_grid_search = GridSearchCV(LR_grid, param_grid=param_grid, cv=cvx ,scoring='accuracy',n_jobs=10)
LR_grid_search.fit(pca_X_train, train_y)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [       nan        nan 0.45230769        nan 0.52369231 0.54769231
 0.54769231 0.72153846 0.76123077 0.74523077        nan        nan
 0.45230769        nan 0.50030769 0.75292308 0.75292308 0.75292308
 0.79230769 0.75292308        nan        nan 0.45230769        nan
 0.52369231 0.848      0.848      0.82430769 0.864      0.832
        nan        nan 0.68246154        nan 0.75292308 0.888
 0.888      0.87230769 0.88       0.88              nan        nan
 0.92              nan 0.92       0.88061538 0.88061538 0.85661538
 0.86461538 0.86461538        nan        nan 0.944             nan
 0.936      0.84061538 0.84061538 0.86461538 0.86461538 0.87261538
        nan        nan 0.944             nan 0.87261538 0.84861538
 0.84861538 0.86461538 0.86461538 0.87261538        nan        nan
 0.90430769        nan 0.87261538 0.85661538 0.85661538 0.84861

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=10, shuffle=True),
             estimator=LogisticRegression(max_iter=1000), n_jobs=10,
             param_grid={'C': [1e-05, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100,
                               1000],
                         'penalty': ['l1', 'l2'],
                         'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag',
                                    'saga']},
             scoring='accuracy')

In [186]:
estimators = [
    ('lr', LR_grid_search.best_estimator_),
    ('svc', svc_grid_search.best_estimator_),
]
clf = StackingClassifier(estimators=estimators, final_estimator=LinearSVC(C=5),n_jobs=10)
clf.fit(pca_X_train, train_y)

StackingClassifier(estimators=[('lr',
                                LogisticRegression(C=1, max_iter=1000,
                                                   penalty='l1',
                                                   solver='liblinear')),
                               ('svc',
                                SVC(C=32, gamma=3.0517578125e-05,
                                    random_state=10))],
                   final_estimator=LinearSVC(C=5), n_jobs=10)

In [187]:
estimators = [
    ('lr', LR_grid_search.best_estimator_),
    ('svc', svc_grid_search.best_estimator_),
]

param_grid = {'final_estimator':[LogisticRegression(C=0.00001),LogisticRegression(C=0.0001),LogisticRegression(C=0.001),LogisticRegression(C=0.01),
                                 LogisticRegression(C=0.1),LogisticRegression(C=1),LogisticRegression(C=10),LogisticRegression(C=100),
                                 LogisticRegression(C=1000)]}

Stacking_grid =StackingClassifier(estimators=estimators,)
Stacking_grid_search = GridSearchCV(Stacking_grid, param_grid=param_grid, cv=cvx ,scoring='accuracy',n_jobs=10)
Stacking_grid_search.fit(pca_X_train, train_y)
Stacking_grid_search.best_estimator_

StackingClassifier(estimators=[('lr',
                                LogisticRegression(C=1, max_iter=1000,
                                                   penalty='l1',
                                                   solver='liblinear')),
                               ('svc',
                                SVC(C=32, gamma=3.0517578125e-05,
                                    random_state=10))],
                   final_estimator=LogisticRegression(C=100))

In [188]:
train_pre_y = cross_val_predict(Stacking_grid_search.best_estimator_, pca_X_train,train_y, cv=cvx)
train_res1=get_measures_gridloo(train_y,train_pre_y)
#     train_res1['PCA']=n_components

test_pre_y = Stacking_grid_search.predict(pca_X_test)
test_res1=get_measures_gridloo(test_y,test_pre_y)

In [189]:
print(train_res1)
print(test_res1)

   TP  TN  FP  FN     Sen     Spe     Acc     PPV     NPV     MCC       AUC
0  64  50   7   5  0.9275  0.8772  0.9048  0.9014  0.9091  0.8076  0.902365
   TP  TN  FP  FN  Sen  Spe  Acc  PPV  NPV  MCC  AUC
0  30  24   0   0  1.0  1.0  1.0  1.0  1.0  1.0  1.0


In [110]:
%%time
################random seed loop#################
train_scores=[]
test_scores=[]
loopn=20
scoring='roc_auc'
np.random.seed(42)
random_states=np.random.choice(range(101), loopn, replace=False)

tmp_train=[]
tmp_test=[]
#print(random_states)
for i in range(loopn):
    train_X,test_X, train_y, test_y = train_test_split(train,
                                                   target,
                                                   test_size = 0.3,
                                                   stratify=target,
                                                   random_state=random_states[i]
                                                   )
    print("train_x.shpae:")
    print(train_X.shape)

    standardScaler = StandardScaler()
    standardScaler.fit(train_X)
    X_standard = standardScaler.transform(train_X)
    X_standard_test = standardScaler.transform(test_X)
    #calculate max n_components
    estimator = PCA(n_components=0.99,random_state=random_states[i])
    pca_X_train = estimator.fit_transform(X_standard)

    n_components=range(10,min(pca_X_train.shape),10)
    print(n_components)
    #n_components=[0.99,0.95,0.90,0.85]
    best_pca_train_scores=[]
    best_pca_test_scores=[]
    for j in n_components:
        estimator = PCA(n_components=j,random_state=random_states[i])
        pca_X_train = estimator.fit_transform(X_standard)
        pca_X_test = estimator.transform(X_standard_test)
        cvx = StratifiedKFold(n_splits=5, shuffle=True, random_state=random_states[i]) 
    
        #scoring = {'AUC': 'roc_auc'}
        cost = [-5, -3, -1, 1, 3, 5, 7, 9, 11, 13, 15]
        gam = [3, 1, -1, -3, -5, -7, -9, -11, -13, -15]
        parameters =[{'kernel': ['rbf'], 'C': [2**x for x in cost],'gamma':[2**x for x in gam]}]

        svc_grid_search=GridSearchCV(estimator=SVC(random_state=random_states[i]),
                                     param_grid=parameters,cv=cvx,scoring=scoring)
        svc_grid_search.fit(pca_X_train, train_y)

        param_grid = {'penalty':['l1', 'l2'],
                      "C":[0.00001,0.0001,0.001, 0.01, 0.1, 1, 10, 100, 1000],
                      "solver":["newton-cg", "lbfgs","liblinear","sag","saga"]
    #               "algorithm":['auto', 'ball_tree', 'kd_tree', 'brute']
                    }
        LR_grid = LogisticRegression(max_iter=1000, random_state=random_states[i])
        LR_grid_search = GridSearchCV(LR_grid, param_grid=param_grid, cv=cvx ,scoring=scoring,n_jobs=10)
        LR_grid_search.fit(pca_X_train, train_y)

        estimators = [
            ('lr', LR_grid_search.best_estimator_),
            ('svc', svc_grid_search.best_estimator_),
        ]
        clf = StackingClassifier(estimators=estimators, 
                                 final_estimator=LinearSVC(C=5, random_state=random_states[i]),n_jobs=10)
        clf.fit(pca_X_train, train_y)

        estimators = [
            ('lr', LR_grid_search.best_estimator_),
            ('svc', svc_grid_search.best_estimator_),
        ]

        param_grid = {'final_estimator':[LogisticRegression(C=0.00001),LogisticRegression(C=0.0001),
                                         LogisticRegression(C=0.001),LogisticRegression(C=0.01),
                                         LogisticRegression(C=0.1),LogisticRegression(C=1),
                                         LogisticRegression(C=10),LogisticRegression(C=100),
                                         LogisticRegression(C=1000)]}

        Stacking_grid =StackingClassifier(estimators=estimators,)
        Stacking_grid_search = GridSearchCV(Stacking_grid, param_grid=param_grid, cv=cvx,
                                            scoring=scoring,n_jobs=10)
        Stacking_grid_search.fit(pca_X_train, train_y)
        Stacking_grid_search.best_estimator_

        train_pre_y = cross_val_predict(Stacking_grid_search.best_estimator_, pca_X_train,train_y, cv=cvx)
        train_res1=get_measures_gridloo(train_y,train_pre_y)

        test_pre_y = Stacking_grid_search.predict(pca_X_test)
        test_res1=get_measures_gridloo(test_y,test_pre_y)
        
        best_pca_train_scores.append(train_res1.loc[:,"AUC"])
        best_pca_test_scores.append(test_res1.loc[:,"AUC"])
    
    train_scores.append(np.max(best_pca_train_scores))
    test_scores.append(best_pca_test_scores[np.argmax(best_pca_train_scores)])
    print("n_components:")
    print(n_components[np.argmax(best_pca_train_scores)])
    
    tmp_train.append(best_pca_train_scores)
    tmp_test.append(best_pca_test_scores)

train_x.shpae:
(233, 3992)
range(10, 208, 10)


KeyboardInterrupt: 

In [105]:
print('mean')
print(np.mean(train_scores))
print(np.mean(test_scores))
print("std")
print(np.std(train_scores))
print(np.std(test_scores))
pd.DataFrame(tmp_train)

mean
0.7493835404327798
0.7397321428571428
std
0.012532381791542108
0.022023953907401497


,0,1,2,3,4,5,6,7,8,9,...,41,42,43,44,45,46,47,48,49,50
0,"0 0.777194 Name: AUC, dtype: float64","0 0.76217 Name: AUC, dtype: float64","0 0.770453 Name: AUC, dtype: float64","0 0.774934 Name: AUC, dtype: float64","0 0.767129 Name: AUC, dtype: float64","0 0.75584 Name: AUC, dtype: float64","0 0.737066 Name: AUC, dtype: float64","0 0.752729 Name: AUC, dtype: float64","0 0.75136 Name: AUC, dtype: float64","0 0.724993 Name: AUC, dtype: float64",...,"0 0.712495 Name: AUC, dtype: float64","0 0.726096 Name: AUC, dtype: float64","0 0.709064 Name: AUC, dtype: float64","0 0.730737 Name: AUC, dtype: float64","0 0.727732 Name: AUC, dtype: float64","0 0.737013 Name: AUC, dtype: float64","0 0.733848 Name: AUC, dtype: float64",None,None,None
1,"0 0.743342 Name: AUC, dtype: float64","0 0.748142 Name: AUC, dtype: float64","0 0.738808 Name: AUC, dtype: float64","0 0.737226 Name: AUC, dtype: float64","0 0.73543 Name: AUC, dtype: float64","0 0.743289 Name: AUC, dtype: float64","0 0.758899 Name: AUC, dtype: float64","0 0.719501 Name: AUC, dtype: float64","0 0.719767 Name: AUC, dtype: float64","0 0.716762 Name: AUC, dtype: float64",...,"0 0.703785 Name: AUC, dtype: float64","0 0.707003 Name: AUC, dtype: float64","0 0.705367 Name: AUC, dtype: float64","0 0.713066 Name: AUC, dtype: float64","0 0.703838 Name: AUC, dtype: float64","0 0.702203 Name: AUC, dtype: float64",None,None,None,None
2,"0 0.732585 Name: AUC, dtype: float64","0 0.746826 Name: AUC, dtype: float64","0 0.72615 Name: AUC, dtype: float64","0 0.749777 Name: AUC, dtype: float64","0 0.743395 Name: AUC, dtype: float64","0 0.758952 Name: AUC, dtype: float64","0 0.737385 Name: AUC, dtype: float64","0 0.730843 Name: AUC, dtype: float64","0 0.738755 Name: AUC, dtype: float64","0 0.715233 Name: AUC, dtype: float64",...,"0 0.733795 Name: AUC, dtype: float64","0 0.727679 Name: AUC, dtype: float64","0 0.730737 Name: AUC, dtype: float64",None,None,None,None,None,None,None
3,"0 0.724621 Name: AUC, dtype: float64","0 0.699517 Name: AUC, dtype: float64","0 0.707375 Name: AUC, dtype: float64","0 0.738701 Name: AUC, dtype: float64","0 0.729261 Name: AUC, dtype: float64","0 0.721403 Name: AUC, dtype: float64","0 0.725937 Name: AUC, dtype: float64","0 0.722879 Name: AUC, dtype: float64","0 0.705474 Name: AUC, dtype: float64","0 0.70542 Name: AUC, dtype: float64",...,"0 0.692656 Name: AUC, dtype: float64","0 0.689651 Name: AUC, dtype: float64","0 0.67573 Name: AUC, dtype: float64",None,None,None,None,None,None,None
4,"0 0.743182 Name: AUC, dtype: float64","0 0.741493 Name: AUC, dtype: float64","0 0.5 Name: AUC, dtype: float64","0 0.759005 Name: AUC, dtype: float64","0 0.743022 Name: AUC, dtype: float64","0 0.7464 Name: AUC, dtype: float64","0 0.744818 Name: AUC, dtype: float64","0 0.746294 Name: AUC, dtype: float64","0 0.729155 Name: AUC, dtype: float64","0 0.735643 Name: AUC, dtype: float64",...,"0 0.712746 Name: AUC, dtype: float64","0 0.708851 Name: AUC, dtype: float64","0 0.707216 Name: AUC, dtype: float64","0 0.699198 Name: AUC, dtype: float64","0 0.711962 Name: AUC, dtype: float64","0 0.705474 Name: AUC, dtype: float64",None,None,None,None
5,"0 0.57817 Name: AUC, dtype: float64","0 0.568251 Name: AUC, dtype: float64","0 0.747823 Name: AUC, dtype: float64","0 0.727306 Name: AUC, dtype: float64","0 0.724301 Name: AUC, dtype: float64","0 0.731893 Name: AUC, dtype: float64","0 0.716337 Name: AUC, dtype: float64","0 0.732159 Name: AUC, dtype: float64","0 0.711856 Name: AUC, dtype: float64","0 0.728835 Name: AUC, dtype: float64",...,"0 0.740177 Name: AUC, dtype: float64","0 0.721296 Name: AUC, dtype: float64","0 0.730684 Name: AUC, dtype: float64","0 0.73696 Name: AUC, dtype: float64","0 0.73063 Name: AUC, dtype: float64","0 0.739964 Name: AUC, dtype: float64",None,None,None,None
6,"0 0.739964 Name: AUC, dtype: float64","0 0.5 Name: AUC, dtype: float64","0 0.708638 Name: AUC, dtype: float64","0 0.71054 Name: AUC, dtype: float64","0 0.727626 Name: AUC, dtype: float64","0 0.723145 Name: AUC, dtype: float64","0

In [106]:
print(train_scores)
print(test_scores)

[0.9295183982683983]
[0    0.940972
Name: AUC, dtype: float64]


In [136]:
random_stat=10
train_X,test_X, train_y, test_y = train_test_split(train,
                                                   target,
                                                   test_size = 0.3,
                                                   stratify=target,
                                                   random_state=random_stat
                                                   )
print("train_x.shpae:")
print(train_X.shape)

standardScaler = StandardScaler()
standardScaler.fit(train_X)
X_standard = standardScaler.transform(train_X)
X_standard_test = standardScaler.transform(test_X)
    #calculate max n_components
estimator = PCA(n_components=0.99,random_state=random_stat)
pca_X_train = estimator.fit_transform(X_standard)

n_components=range(10,min(pca_X_train.shape),10)
print(n_components)
    #n_components=[0.99,0.95,0.90,0.85]
best_pca_train_scores=[]
best_pca_test_scores=[]
for j in n_components:
    estimator = PCA(n_components=j,random_state=random_stat)
    pca_X_train = estimator.fit_transform(X_standard)
    pca_X_test = estimator.transform(X_standard_test)
    cvx = StratifiedKFold(n_splits=5, shuffle=True, random_state=random_stat) 
    
        #scoring = {'AUC': 'roc_auc'}
    cost = [-5, -3, -1, 1, 3, 5, 7, 9, 11, 13, 15]
    gam = [3, 1, -1, -3, -5, -7, -9, -11, -13, -15]
    parameters =[{'kernel': ['rbf'], 'C': [2**x for x in cost],'gamma':[2**x for x in gam]}]

    svc_grid_search=GridSearchCV(estimator=SVC(random_state=random_stat),
                                 param_grid=parameters,cv=cvx,scoring=scoring)
    svc_grid_search.fit(pca_X_train, train_y)

    param_grid = {'penalty':['l1', 'l2'],
                  "C":[0.00001,0.0001,0.001, 0.01, 0.1, 1, 10, 100, 1000],
                  "solver":["newton-cg", "lbfgs","liblinear","sag","saga"]
    #               "algorithm":['auto', 'ball_tree', 'kd_tree', 'brute']
                  }
    LR_grid = LogisticRegression(max_iter=1000, random_state=random_stat)
    LR_grid_search = GridSearchCV(LR_grid, param_grid=param_grid, cv=cvx ,scoring=scoring,n_jobs=10)
    LR_grid_search.fit(pca_X_train, train_y)

    estimators = [
        ('lr', LR_grid_search.best_estimator_),
        ('svc', svc_grid_search.best_estimator_),
    ]
    clf = StackingClassifier(estimators=estimators, 
                             final_estimator=LinearSVC(C=5, random_state=random_stat),n_jobs=10)
    clf.fit(pca_X_train, train_y)

    estimators = [
        ('lr', LR_grid_search.best_estimator_),
        ('svc', svc_grid_search.best_estimator_),
    ]

    param_grid = {'final_estimator':[LogisticRegression(C=0.00001),LogisticRegression(C=0.0001),
                                    LogisticRegression(C=0.001),LogisticRegression(C=0.01),
                                    LogisticRegression(C=0.1),LogisticRegression(C=1),
                                    LogisticRegression(C=10),LogisticRegression(C=100),
                                    LogisticRegression(C=1000)]}

    Stacking_grid =StackingClassifier(estimators=estimators,)
    Stacking_grid_search = GridSearchCV(Stacking_grid, param_grid=param_grid, cv=cvx,
                                        scoring=scoring,n_jobs=10)
    Stacking_grid_search.fit(pca_X_train, train_y)
    Stacking_grid_search.best_estimator_

    train_pre_y = cross_val_predict(Stacking_grid_search.best_estimator_, pca_X_train,train_y, cv=cvx)
    train_res1=get_measures_gridloo(train_y,train_pre_y)

    test_pre_y = Stacking_grid_search.predict(pca_X_test)
    test_res1=get_measures_gridloo(test_y,test_pre_y)
        
    best_pca_train_scores.append(train_res1.loc[:,"AUC"])
    best_pca_test_scores.append(test_res1.loc[:,"AUC"])

train_x.shpae:
(630, 4958)
range(10, 459, 10)


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [       nan        nan 0.5               nan 0.5        0.72752591
 0.72752591 0.72722357 0.72722357 0.72727396        nan        nan
 0.5               nan 0.5        0.7279292  0.7279292  0.72183167
 0.72208364 0.72203325        nan        nan 0.77937848        nan
 0.77937848 0.76784054 0.76784054 0.7294922  0.73946975 0.73427939
        nan        nan 0.72651724        nan 0.73221161 0.80780132
 0.80780132 0.75398314 0.74395514 0.73674903        nan        nan
 0.79611033        nan 0.73684961 0.81747656 0.81747656 0.79812631
 0.74435829 0.73695063        nan        nan 0.81656957        nan
 0.73659787 0.8178797  0.8178797  0.81475547 0.74425751 0.73715221
        nan        nan 0.81803089        nan 0.73710182 0.81808128
 0.81808128 0.81787972 0.7443079  0.73715221        nan        nan
 0.81813167        nan 0.73715221 0.81818208 0.818182

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [       nan        nan 0.5               nan 0.5        0.76002835
 0.76002835 0.75977641 0.7598268  0.75987719        nan        nan
 0.5               nan 0.5        0.76637778 0.76637778 0.76209437
 0.76214479 0.76199361        nan        nan 0.77937848        nan
 0.77937848 0.7980747  0.7980747  0.77615342 0.77988248 0.77726207
        nan        nan 0.73468066        nan 0.73926649 0.81339317
 0.81339317 0.76461244 0.76622523 0.77091203        nan        nan
 0.8031139         nan 0.77207165 0.81873463 0.81868424 0.76667876
 0.76264722 0.76950097        nan        nan 0.81868413        nan
 0.76965223 0.81334212 0.81334212 0.7999883  0.76219367 0.76945058
        nan        nan 0.81203169        nan 0.76960175 0.8118301
 0.8118301  0.81001608 0.76214328 0.76945058        nan        nan
 0.8119813         nan 0.76945058 0.81177974 0.8118301

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [       nan        nan 0.5               nan 0.5        0.76023005
 0.76023005 0.75992768 0.75997807 0.75997807        nan        nan
 0.5               nan 0.5        0.76693226 0.76693226 0.76315279
 0.76320318 0.76320318        nan        nan 0.77937848        nan
 0.77937848 0.79172543 0.79172543 0.77141677 0.77484355 0.773231
        nan        nan 0.73609188        nan 0.73997216 0.78441832
 0.78436791 0.74027352 0.75604637 0.76506705        nan        nan
 0.77534727        nan 0.77045919 0.76849339 0.768443   0.7132626
 0.74979782 0.76395833        nan        nan 0.76395785        nan
 0.76476468 0.76168971 0.76168971 0.72601139 0.74899152 0.76360562
        nan        nan 0.75236669        nan 0.76350483 0.74979672
 0.74979672 0.74848628 0.74884034 0.76360562        nan        nan
 0.74762978        nan 0.76360562 0.74757935 0.74752896 

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [       nan        nan 0.5               nan 0.5        0.76375745
 0.76375745 0.76360631 0.76355592 0.7636567         nan        nan
 0.5               nan 0.5        0.77187081 0.77187081 0.76804086
 0.7677889  0.76758735        nan        nan 0.77937848        nan
 0.77937848 0.80644012 0.80644012 0.78708893 0.79071732 0.78915513
        nan        nan 0.73609188        nan 0.73982097 0.79610922
 0.79610922 0.7563992  0.77353291 0.78068888        nan        nan
 0.79686516        nan 0.78865112 0.75927116 0.75927116 0.71124706
 0.76894714 0.7790763         nan        nan 0.74400175        nan
 0.78109205 0.71850287 0.71850286 0.674661

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [       nan        nan 0.5               nan 0.5        0.76365684
 0.76365684 0.76365682 0.76365682 0.76365682        nan        nan
 0.5               nan 0.5        0.77101407 0.77101407 0.76799048
 0.76819203 0.76804085        nan        nan 0.77937848        nan
 0.77937848 0.80553323 0.80553323 0.78406558 0.78779462 0.7853254
        nan        nan 0.73609188        nan 0.73987138 0.78572927
 0.78572927 0.7485393  0.76995572 0.78008468        nan        nan
 0.79802428        nan 0.79187631 0.75060556 0.75060556 0.7106441
 0.76511803 0.77811935        nan        nan 0.73599157        nan
 0.78043749 0.71578355 0.71578355 0.67869433 0.76476531 0.77771619
        nan        nan 0.70515125        nan 0.77786736 0.70263078
 0.70263078 0.66584344 0.76476531 0.77776658        nan        nan
 0.67531638        nan 0.77781697 0.69819614 0.69829692

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [       nan        nan 0.5               nan 0.5        0.76431185
 0.76431185 0.76431185 0.76426146 0.76431185        nan        nan
 0.5               nan 0.5        0.77222344 0.77222344 0.76824242
 0.76814164 0.76804086        nan        nan 0.77937848        nan
 0.77937848 0.80890938 0.80890938 0.7885504  0.79248104 0.7903141
        nan        nan 0.73609188        nan 0.73987138 0.80513009
 0.8050797  0.77323107 0.78396456 0.78996155        nan        nan
 0.78769393        nan 0.78764315 0.77363468 0.77363468 0.73589007
 0.77963094 0.78910484        nan        nan 0.74692633        nan
 0.79041503 0.75241876 0.75241876 0.71527897 0.77877426 0.78875209
        nan        nan 0.71235541        nan 0.78905444 0.74359976
 0.74349896 0.70741763 0.77867348 0.78880248        nan        nan
 0.68846926        nan 0.78895367 0.7382076  0.7383083

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [       nan        nan 0.5               nan 0.5        0.76375743
 0.76375743 0.76355588 0.76355588 0.76355588        nan        nan
 0.5               nan 0.5        0.77030851 0.77030851 0.76703301
 0.76713379 0.767033          nan        nan 0.77937848        nan
 0.77937848 0.80538177 0.80538177 0.7865348  0.79031426 0.78759301
        nan        nan 0.73609188        nan 0.73987138 0.79943596
 0.79943596 0.76350561 0.77917742 0.78542611        nan        nan
 0.77852217        nan 0.78723995 0.76355705 0.76355705 0.72243596
 0.77459172 0.78406552        nan        nan 0.73629313        nan
 0.78718986 0.74914407 0.74909368 0.71089559 0.77403742 0.78371274
        nan        nan 0.71114649        nan 0.78416629 0.74329824
 0.74339903 0.70651108 0.77393664 0.78376313        nan        nan
 0.68509251        nan 0.78376313 0.74022421 0.740022

In [135]:
print(np.max(best_pca_train_scores))
print(best_pca_test_scores[np.argmax(best_pca_train_scores)])
print("n_components:")
print(n_components[np.argmax(best_pca_train_scores)])

print(best_pca_train_scores)
print(best_pca_test_scores)

0.755472466338789
0    0.77037
Name: AUC, dtype: float64
n_components:
40
[0    0.720632
Name: AUC, dtype: float64, 0    0.543195
Name: AUC, dtype: float64, 0    0.752378
Name: AUC, dtype: float64, 0    0.755472
Name: AUC, dtype: float64, 0    0.752308
Name: AUC, dtype: float64, 0    0.750786
Name: AUC, dtype: float64, 0    0.725379
Name: AUC, dtype: float64, 0    0.733331
Name: AUC, dtype: float64, 0    0.720652
Name: AUC, dtype: float64, 0    0.71901
Name: AUC, dtype: float64, 0    0.711179
Name: AUC, dtype: float64, 0    0.711189
Name: AUC, dtype: float64, 0    0.727032
Name: AUC, dtype: float64, 0    0.738148
Name: AUC, dtype: float64, 0    0.71907
Name: AUC, dtype: float64, 0    0.71911
Name: AUC, dtype: float64, 0    0.707984
Name: AUC, dtype: float64, 0    0.717528
Name: AUC, dtype: float64, 0    0.731758
Name: AUC, dtype: float64, 0    0.720622
Name: AUC, dtype: float64, 0    0.71906
Name: AUC, dtype: float64, 0    0.730156
Name: AUC, dtype: float64, 0    0.738067
Name: AUC, dt